# Initial imports

In [ ]:
!python --version

In [ ]:
!pip install -U scikit-learn==1.2.1

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
!pip install numpy==1.22.0

In [ ]:

import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
print(np.__version__)

In [ ]:
# import required packages

# import spacy
import pandas as pd
import json
import random
import re
import string
#import nltk
from collections import Counter

#from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#from nltk.corpus import stopwords
#nltk.download('stopwords')

from tqdm import tqdm

# Data for Classification

Load the data for classification

In [ ]:
path = './'

In [ ]:
data_for_classifier = pd.read_csv(path+'allData.csv')

In [ ]:
# check first 5 rows

data_for_classifier.head()

In [ ]:
# check label distribution

data_for_classifier['label'].value_counts(normalize=True)

In [ ]:
# Calculate lengths of spans
data_for_classifier['Text_snippet_len'] = data_for_classifier['Text_snippet'].apply(len)

In [ ]:
# Find minimum, maximum, and mean lengths - update column name to text column of the df
min_length = data_for_classifier['Text_snippet_len'].min()
max_length = data_for_classifier['Text_snippet_len'].max()
mean_length = data_for_classifier['Text_snippet_len'].mean()
median_length = data_for_classifier['Text_snippet_len'].median()

# Print results
print(f"Minimum Length: {min_length}")
print(f"Maximum Length: {max_length}")
print(f"Mean Length: {mean_length}")
print(f"Median Length: {median_length}")

# Split the data into train and test (80:20)

In [ ]:
training_data, testing_data = train_test_split(data_for_classifier, test_size=0.2, random_state=1)

In [ ]:
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

In [ ]:
training_data.head()

In [ ]:
# check label counts

training_data['label'].value_counts()

In [ ]:
# check label percentages

training_data['label'].value_counts(normalize=True)

In [ ]:
# check first 5 rows

testing_data.head()

In [ ]:
# save test data as a separate file for use later

testing_data.to_csv('test_data.csv')

In [ ]:
# check label counts

testing_data['label'].value_counts()

In [ ]:
# check label percentages

testing_data['label'].value_counts(normalize=True)

# BERT Classification 1

Run the second BERT classification below for confidence intervals etc. 

Install and import BERT specific packages

In [ ]:
!pip install - q tensorflow

In [ ]:
# check for GPU using tensorflow. If present it will say something like Found GPU at: /device:GPU:0

import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
!pip install - q torch

In [ ]:
# check for GPU using torch. If present it will say something like There are 1 GPU(s) available. We will use the GPU: Tesla T4

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip install - q transformers==4.5.0

In [ ]:
!pip install transformers

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import StratifiedKFold
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset
from tqdm import trange

from transformers import BertForSequenceClassification, BertTokenizerFast, BertTokenizer, AdamW, AutoModel, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

In [ ]:
import torch.nn as nn

In [ ]:
tf.__version__

In [ ]:
torch.__version__

FOR bert_base_uncased

In [ ]:
BERT_tokenizer = 'bert-base-uncased'

MAX_TKN_LEN=511

In [ ]:
# import BERT-base pretrained model

trained_bert_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
trained_bert_model.cuda()

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')



In [ ]:
training_data.head()

In [ ]:
training_data.shape

In [ ]:
text_col = 'Text_snippet'
label_col = 'label'


In [ ]:
training_data['Text_snippet'] = training_data['Text_snippet'].apply(lambda x: x.strip().lower())
training_data['Text_snippet']= training_data['Text_snippet'].str.replace('[^\w\s]','')
print('num annotations:', len(training_data), '\n\n', training_data['label'].value_counts(), '\n\n', training_data[['label', 'Text_snippet']].head(5))

# BERT functions

In [ ]:
# Create the DataLoaders for our training and validation sets.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.
def create_dataloader(train_dataset, val_dataset, batch_size=16):
    # We'll take training samples in random order.
    train_dataloader = DataLoader(
        train_dataset,  # The training samples.
        sampler=RandomSampler(train_dataset),  # Select batches randomly
        batch_size=batch_size  # Trains with this batch size.
    )
    # For validation the order doesn't matter, so we'll just read them sequentially.
    validation_dataloader = DataLoader(
        val_dataset,  # The validation samples.
        sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
        batch_size=batch_size  # Evaluate with this batch size.
    )

    return [train_dataloader, validation_dataloader]

In [ ]:
# convert labels to numerical values
def convert_to_cat(labels, binary=False):
    if not isinstance(labels, pd.Series):
        labels = pd.Series(labels)
    if binary:
        main_val = labels.mode()[0]
        other_val = [x for x in labels.unique() if x != main_val]
        labels = np.where(labels != main_val, 1, 0)
        corresp = pd.DataFrame([main_val, 'other'], columns=['old_label'])
    elif np.issubdtype(labels.dtype, np.number) and (labels.min() >= 0):
        print('labels already in a good format')
        return {'labels': labels, 'categories': pd.DataFrame(labels.unique())}
    else:
        cats = labels.astype('category').cat
        labels = cats.codes.astype('long')  # convert annotations to integers
        corresp = pd.DataFrame(cats.categories, columns=['old_label'])
    corresp.index.rename('new_label', inplace=True)
    print('labels have been transformed for the model:\n\n', corresp)
    return {'labels': labels, 'categories': corresp}

In [ ]:
# Function to calculate performance of our predictions vs labels
def perf_metrics(preds, labels, average='weighted', debug=False):
    try:
        pred_flat = np.argmax(preds, axis=1).flatten()
        # pred_flat = torch.max(pred_vec, 1)[1]
    except:
        print('only 1 dimension in labels prediction, no need for argmax')
        pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    acc = accuracy_score(labels_flat, pred_flat)
    f1 = f1_score(labels_flat, pred_flat, average=average)
    p = precision_score(labels_flat, pred_flat, average=average)
    r = recall_score(labels_flat, pred_flat, average=average)
    if debug:
        print("PERF -- Acc: {:.3f} F1: {:.3f} Precision: {:.3f} Recall: {:.3f}".format(acc, f1, p, r))
    return {'f1': f1, 'acc': acc, 'p': p, 'r': r}


def perf_metrics_classes(preds, labels):
    try:
        pred_flat = np.argmax(preds, axis=1).flatten()
    except:
        print('only 1 dimension in labels prediction, no need for argmax')
        pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    report = classification_report(labels_flat, pred_flat, output_dict=True)
    df = pd.DataFrame(report).sort_index().transpose()
    # df['accuracy'] = accuracy_score(labels, preds)

    return df

In [ ]:
# functions to handle different devices
def to_cpu(vec, detach=True):
    try:
        vec = vec.detach().cpu().numpy() if detach else vec.to('cpu').numpy()
    except:
        vec = vec.detach().numpy() if detach else vec.numpy()
    return vec


def model_to_cpu(model):
    try:
        model.to("cpu")
    except:
        pass


def batch_to_gpu(batch, device=None):
    if device is None:
        return batch
    try:  # we're using a GPU
        batch = tuple(t.to(device) for t in batch)
    except:
        batch = tuple(t for t in batch)
    return batch

def get_device():
    # specify GPU device
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if device != 'cpu':
            print('number GPUs used:', torch.cuda.device_count())
            print('device name:', torch.cuda.get_device_name(0))
    except:
        device = None
        print('no CUDA capable device detected')
    return device

In [ ]:
def prep_BERT_dataset(sentences, labels=None, BERT_tokenizer='bert-base-uncased', MAX_TKN_LEN=511, debug=False):
    """
    JC: change tokenizer to bert-base-uncased or 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
    
    prepares data for BERT fine-tuning
    :param sentences: list or array-like. list of texts to classify
    :param labels: list or array-like, default None. list of labels corresponding to sentences
    :param BERT_tokenizer: string, default 'bert-base-uncased'. BERT base model used
    :param MAX_TKN_LEN: integer, default 511. see https://github.com/huggingface/transformers/issues/2446
    :param debug: bool, default False. set to True to display inetrmediate results
    :return: prepared dataset (torch Dataset) and nmber of dictinct labels
    """
    sentences = pd.Series(sentences)
    # load relevant data and add special tokens for BERT to work properly
    sentences = ["[CLS] " + query + " [SEP]" for query in sentences]
    if labels is not None:
        labels = convert_to_cat(labels, binary=False)['labels']
    else:
        labels = pd.Series([1] * len(sentences))
    if debug: print(sentences[0])

    # Tokenize with BERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(BERT_tokenizer, do_lower_case=True)
    if MAX_TKN_LEN is not None:
        print('cutting the length of tokens to', MAX_TKN_LEN)
        tokenized_texts = [tokenizer.tokenize(sent)[0:MAX_TKN_LEN] for sent in sentences]
    else:
        tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]
    if debug:
        print("Tokenize the first sentence:")
        print(len(tokenized_texts), len(input_ids), len(input_ids[0]))
        print(tokenized_texts[0], input_ids[0])

    # add paddding to input_ids
    input_ids_padded = pad_sequence([torch.tensor(i) for i in input_ids]).transpose(0, 1)
    if debug: print(input_ids_padded.size(), len(input_ids_padded))

    # Create attention masks
    attention_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids_padded:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    # create dataset
    dataset = TensorDataset(input_ids_padded, torch.tensor(attention_masks), torch.tensor(labels))
    num_labels = labels.nunique()
    return {'dataset': dataset, 'num_labels': num_labels}

In [ ]:
def finetune_BERT_pytorch(model, train_dataloader, validation_dataloader, n_epochs=3, output_dir='./bert_base_v2_Jun26/'):
    """
    JC: update output_dir or say None
    
    fine-tunes Bert for text classification
    :param model: base Bert model
    :param train_dataloader: Tensor Dataset. training data
    :param validation_dataloader: Tensor Dataset. testing data
    :param n_epochs: integer, default 5. number of epochs
    :param output_dir: string, default None. directory where to save the fine-tuned model
    :return:    model: fine-tuned Bert model with highest performance over k folds
                stats: high level performance statistics
                stats_classes: detailed performance statistics
    """
    ###################################################################################
    # BERT fine-tuning parameters
    device = get_device()
    try:
        model.cuda()
    except:
        device = None
        print('using CPU, this will be slow!')
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)

    # BERT training loop
    train_loss_set = []
    best_f1, best_epoch = 0, 0
    for _ in trange(n_epochs, desc="Epoch"):
        ###################################################################################
        ## TRAINING

        # Set our model to training mode
        model.train()
        # Tracking variables
        tr_loss, tr_perf, tr_perf_classes = 0, Counter({}), pd.DataFrame()
        running_len = 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs['loss'], outputs['logits']
            train_loss_set.append(loss.item())
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            tmp_tr_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_tr_perf.update((k, v * len(b_input_ids)) for k, v in tmp_tr_perf.items())
            running_len += len(b_input_ids)
            tr_perf = tr_perf + Counter(tmp_tr_perf)
            tmp_tr_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            tr_perf_classes = pd.concat((tr_perf_classes, tmp_tr_perf_classes))

        # print('classes detail \n\n', tr_perf_classes)
        tr_perf = {k: v / running_len for k, v in tr_perf.items()}
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[
            ['f1-score', 'precision', 'recall']].multiply(tr_perf_classes['support'], axis="index")
        tr_perf_classes = tr_perf_classes.groupby(tr_perf_classes.index).sum()
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[['f1-score', 'precision', 'recall']].div(
            tr_perf_classes['support'], axis="index")
        print('TRAIN - Loss: {:.3f} - F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(tr_loss / (1 + step),
                                                                                         tr_perf['f1'], tr_perf['acc'],
                                                                                         tr_perf['p'], tr_perf['r']))

        ###################################################################################
        ## VALIDATION

        # Put model in evaluation mode
        model.eval()
        # Tracking variables
        eval_perf, eval_perf_classes = Counter({}), pd.DataFrame()
        running_len = 0
        # Evaluate data for one epoch
        for step, batch in enumerate(validation_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():  # Telling the model not to compute or store gradients, saving memory and speeding up validation
                # Forward pass, calculate logit predictions
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
                loss, logits = outputs['loss'], outputs['logits']
            # Update tracking variables
            tmp_eval_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_eval_perf.update((k, v * len(b_input_ids)) for k, v in tmp_eval_perf.items())
            # print('STEP:', step, 'LEN', len(b_input_ids), tmp_eval_perf)
            running_len += len(b_input_ids)
            eval_perf = eval_perf + Counter(tmp_eval_perf)
            tmp_eval_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            eval_perf_classes = pd.concat((eval_perf_classes, tmp_eval_perf_classes))

        eval_perf = {k: v / running_len for k, v in
                     eval_perf.items()}  # eval_perf = {k:v/(1+step) for k,v in eval_perf.items()}
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[
            ['f1-score', 'precision', 'recall']].multiply(eval_perf_classes['support'], axis="index")
        eval_perf_classes = eval_perf_classes.groupby(eval_perf_classes.index).sum()
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[
            ['f1-score', 'precision', 'recall']].div(eval_perf_classes['support'], axis="index")
        print('TEST -- F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(eval_perf['f1'], eval_perf['acc'],
                                                                          eval_perf['p'], eval_perf['r']))

        # store perf metrics and model
        if eval_perf['f1'] >= best_f1:
            best_f1 = eval_perf['f1']
            best_epoch = _ + 1
            stats_to_save = eval_perf
            tr_perf_classes['dataset'] = 'train'
            eval_perf_classes['dataset'] = 'test'
            stats_classes_to_save = pd.concat([tr_perf_classes, eval_perf_classes])

        if output_dir is not None: #save model with best F1
            try:
                print('saving...')
                model.save_pretrained(output_dir)
                stats_classes_to_save.to_csv(output_dir + '/stats.csv', header=True)
            except:
                print('model not saved, please enter valid path')
        print('best F1 score obtained: {:.3f} at epoch {}'.format(best_f1, best_epoch))

    return {'stats': stats_to_save, 'stats_classes': stats_classes_to_save, 'model': model}

In [ ]:
def run_BERT(model, train_dataloader, validation_dataloader, n_epochs=3, output_dir='./bert_base_v2_Jun26/'):
    """
    JC: update output_dir or say None
    JC: This is where we can implement weighted crossentropy loss for class imbalance
    
    fine-tunes Bert for text classification
    :param model: base Bert model
    :param train_dataloader: Tensor Dataset. training data
    :param validation_dataloader: Tensor Dataset. testing data
    :param n_epochs: integer, default 5. number of epochs
    :param output_dir: string, default None. directory where to save the fine-tuned model
    :return:    model: fine-tuned Bert model with highest performance over k folds
                stats: high level performance statistics
                stats_classes: detailed performance statistics
    """
    ###################################################################################
    # BERT fine-tuning parameters
    device = get_device()
    try:
        model.cuda()
    except:
        #device = None
        print('could not move Bert to Cuda, using CPU, this will be slow!')
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
    #loss_fn = nn.CrossEntropyLoss() #JC try this?
    
    # weighted cross entropy loss for class imbalance
    class_weights = torch.tensor([0.82, 0.17], device=device)  # Modify these weights accordingly [wt of class0, wt of class1] - To TRY with lgbtq
    loss_fn = nn.CrossEntropyLoss(weight=class_weights) # To TRY with lgbtq
    
    # BERT training loop
    train_loss_set = []
    best_f1, best_epoch = 0, 0
    for _ in trange(n_epochs, desc="Epoch"):
        ###################################################################################
        ## TRAINING

        # Set our model to training mode
        model.train()
        # Tracking variables
        tr_loss, tr_perf, tr_perf_classes = 0, Counter({}), pd.DataFrame()
        running_len = 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            #loss, logits = outputs['loss'], outputs['logits']
            #train_loss_set.append(loss.item()) 
            
            #if using cross entropy then comment out loss, logits and use the lines below
            ##loss = output[0] #keep this commented
            
            logits = outputs[1]
            loss = loss_fn(logits,b_labels)
            print(loss)
            train_loss_set.append(loss.item())
            
            
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            tmp_tr_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_tr_perf.update((k, v * len(b_input_ids)) for k, v in tmp_tr_perf.items())
            running_len += len(b_input_ids)
            tr_perf = tr_perf + Counter(tmp_tr_perf)
            tmp_tr_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            tr_perf_classes = pd.concat((tr_perf_classes, tmp_tr_perf_classes))

        # print('classes detail \n\n', tr_perf_classes)
        tr_perf = {k: v / running_len for k, v in tr_perf.items()}
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[
            ['f1-score', 'precision', 'recall']].multiply(tr_perf_classes['support'], axis="index")
        tr_perf_classes = tr_perf_classes.groupby(tr_perf_classes.index).sum()
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[['f1-score', 'precision', 'recall']].div(
            tr_perf_classes['support'], axis="index")
        print('TRAIN - Loss: {:.3f} - F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(tr_loss / (1 + step),
                                                                                         tr_perf['f1'], tr_perf['acc'],
                                                                                         tr_perf['p'], tr_perf['r']))

        ###################################################################################
        ## VALIDATION

        # Put model in evaluation mode
        model.eval()
        # Tracking variables
        eval_perf, eval_perf_classes = Counter({}), pd.DataFrame()
        running_len = 0
        # Evaluate data for one epoch
        for step, batch in enumerate(validation_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():  # Telling the model not to compute or store gradients, saving memory and speeding up validation
                # Forward pass, calculate logit predictions
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
                loss, logits = outputs['loss'], outputs['logits']
            # Update tracking variables
            tmp_eval_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_eval_perf.update((k, v * len(b_input_ids)) for k, v in tmp_eval_perf.items())
            # print('STEP:', step, 'LEN', len(b_input_ids), tmp_eval_perf)
            running_len += len(b_input_ids)
            eval_perf = eval_perf + Counter(tmp_eval_perf)
            tmp_eval_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            eval_perf_classes = pd.concat((eval_perf_classes, tmp_eval_perf_classes))

        eval_perf = {k: v / running_len for k, v in
                     eval_perf.items()}  # eval_perf = {k:v/(1+step) for k,v in eval_perf.items()}
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[
            ['f1-score', 'precision', 'recall']].multiply(eval_perf_classes['support'], axis="index")
        eval_perf_classes = eval_perf_classes.groupby(eval_perf_classes.index).sum()
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[
            ['f1-score', 'precision', 'recall']].div(eval_perf_classes['support'], axis="index")
        print('TEST -- F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(eval_perf['f1'], eval_perf['acc'],
                                                                          eval_perf['p'], eval_perf['r']))

        # store perf metrics and model
        if eval_perf['f1'] >= best_f1:
            best_f1 = eval_perf['f1']
            best_epoch = _ + 1
            stats_to_save = eval_perf
            tr_perf_classes['dataset'] = 'train'
            eval_perf_classes['dataset'] = 'test'
            stats_classes_to_save = pd.concat([tr_perf_classes, eval_perf_classes])

        if output_dir is not None: #save model with best F1
            try:
                print('saving...')
                model.save_pretrained(output_dir)
                stats_classes_to_save.to_csv(output_dir + '/stats.csv', header=True)
            except:
                print('model not saved, please enter valid path')
        print('best F1 score obtained: {:.3f} at epoch {}'.format(best_f1, best_epoch))

    return {'stats': stats_to_save, 'stats_classes': stats_classes_to_save, 'model': model}

In [ ]:
def train_BERT(sentences, labels, BERT_tokenizer='bert-base-uncased', test_size=0.1,
                        n_epochs=20, batch_size=16, output_dir='./bert_base_v2_Jun26/', MAX_TKN_LEN=511):
    """
    JC: update output_dir or say None; 
    JC: update tokenizer to bert-base-uncased or 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
    JC: Upped epochs from 3 to 20 to facilitate improved generalisability on CRIS data
    
    formats dataset and fine-tunes Bert model on it
    :param sentences: list or array-like. list of texts to classify
    :param labels: list or array-like. list of labels corresponding to sentences
    :param BERT_tokenizer: string, default 'bert-base-uncased'. BERT base model used
    :param test_size: float, default 0.10. train/test split size
    :param n_epochs: integer, default 5. number of epochs used to fine-tune Bert
    :param batch_size: integer, default 32. how many samples pper batch to load
    :param output_dir: string, default None. directory where to save the fine-tuned model
    :param MAX_TKN_LEN: integer, default 511. see https://github.com/huggingface/transformers/issues/2446
    :return:
    """
    print('formating dataset')
    prep_data = prep_BERT_dataset(sentences=sentences, labels=labels, BERT_tokenizer=BERT_tokenizer,
                                  MAX_TKN_LEN=MAX_TKN_LEN)
    dataset = prep_data['dataset']
    num_labels = prep_data['num_labels']
    # split into train/test
    print('splitting in train/test sets')
    test_len = int(len(dataset) * test_size)
    train_len = len(dataset) - test_len
    print('test set:', test_len, 'train set:', train_len)
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, test_len])
    # Create the DataLoaders for our training and validation sets.
    train_dataloader, validation_dataloader = create_dataloader(train_dataset, val_dataset, batch_size=batch_size)
    # Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
    print('loading pre-trained BERT')
    pretrained_model = BertForSequenceClassification.from_pretrained(BERT_tokenizer, num_labels=num_labels, output_attentions = False, output_hidden_states = False)
    # train and evaluate BERT
    print('training BERT')
    res = finetune_BERT_pytorch(pretrained_model, train_dataloader, validation_dataloader, n_epochs=n_epochs, output_dir=output_dir)
    return res

In [ ]:
# load pre-trained model and classify a new sentence
def load_and_run_BERT(sentences, trained_bert_model, BERT_tokenizer='bert-base-uncased', MAX_TKN_LEN=511,
                      batch_size=16):
    """
    JC: update tokenizer to bert-base-uncased or 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
    
    loads pre-trained Bert model and runs it on new text to classify
    :param sentences: list or array-like. list of texts to classify
    :param trained_bert_model: string or pytorch model. pre-trained model name or path
    :param BERT_tokenizer: string, default 'bert-base-uncased'. BERT base model used
    :param MAX_TKN_LEN: integer, default 511. see https://github.com/huggingface/transformers/issues/2446
    :param batch_size: integer, default 32. how many samples pper batch to load
    :return: dataframe of sentences classified along with probaility scores
    """
    if isinstance(trained_bert_model, str):  # load trained BERT model if needed
        trained_bert_model = BertForSequenceClassification.from_pretrained(trained_bert_model)
    preds_class, probs, preds = [], [], pd.DataFrame()
    sentences = pd.Series(sentences)
    sentences_dataset = \
    prep_BERT_dataset(sentences, labels=None, BERT_tokenizer=BERT_tokenizer, MAX_TKN_LEN=MAX_TKN_LEN)['dataset']
    # b_input_ids, b_input_mask, b_labels = sentences_dataset.tensors
    validation_dataloader = DataLoader(sentences_dataset, sampler=SequentialSampler(sentences_dataset),
                                       batch_size=batch_size)
    for step, batch in enumerate(validation_dataloader):
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = trained_bert_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs['loss'], outputs['logits']
        preds_curr = logits.detach().numpy()
        preds = preds.append(pd.DataFrame(preds_curr), ignore_index=True)
        probs = np.append(probs, np.max(np.exp(preds_curr) / (1 + np.exp(preds_curr)), axis=1))
        preds_class = np.append(preds_class, np.argmax(preds_curr, axis=1).flatten())

    # put results in nice format
    preds['sentences'] = sentences
    preds['preds'] = preds_class
    preds['probs'] = probs
    preds['label_goldstd'] = df['label'] #testing something - remove this if not needed or causing errors - use only when you want to do error analysis
    preds['brcid'] = df['brcid'] #testing something - remove this if not needed or causing errors - use only when you want to do error analysis
    preds['CN_Doc_ID'] = df['CN_Doc_ID'] #testing something - remove this if not needed or causing errors - use only when you want to do error analysis
    return preds


In [ ]:
# TO RUN K-FOLD VALIDATION
def run_KFOLD(dataset, base_model, model_trainer, base_model_loader=None,
             n_splits=10, random_state=42, n_epochs=3, **kwargs):
      
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    labels = pd.Series([1] * len(dataset))
    # tracking variables
    best_f1, fold_nb = 0, 0
    stats, stats_classes = pd.DataFrame(), pd.DataFrame()
    # run k fold
    for train_ix, test_ix in kfold.split(labels, labels):
        fold_nb +=1
        # need to load each time otherwise remembers training from previous fold
        if base_model_loader is not None:
            base_model_tmp = base_model_loader(base_model, **kwargs)
        else:
            base_model_tmp = base_model
        print('####################### RUNNING FOLD:', fold_nb)
        train_dataset = torch.utils.data.Subset(dataset, train_ix)
        val_dataset = torch.utils.data.Subset(dataset, test_ix)
        print(type(train_dataset), ' train set:', len(train_dataset), ' test set:',len(val_dataset))
        train_dataloader, validation_dataloader = create_dataloader(train_dataset, val_dataset)
        print('training and evaluating model')
        res = model_trainer(base_model_tmp, train_dataloader, validation_dataloader, n_epochs=n_epochs, output_dir=None)
        del base_model_tmp
        # store perf metrics and model
        stats_tmp = pd.DataFrame.from_dict(res['stats'],orient='index', columns=['value'])
        stats_tmp['fold'] = fold_nb
        stats = pd.concat([stats, stats_tmp])
        res['stats_classes']['fold'] = fold_nb
        stats_classes = pd.concat([stats_classes, res['stats_classes']])
        if res['stats']['f1'] >= best_f1:
            best_f1 = res['stats']['f1']
            res_to_save = res
            
        
    print('best F1 score obtained across splits: {:.3f}'.format(best_f1))
    return {'stats':stats, 'stats_classes':stats_classes, 'model':res_to_save['model']}

In [ ]:
def BERT_KFOLD(sentences, labels, BERT_tokenizer='bert-base-uncased',
               n_splits=10, random_state=42, n_epochs=3, output_dir='./bert_base_v2_Jun26/', MAX_TKN_LEN=511):
    """
    JC: update output_dir or say None; 
    JC: update tokenizer to bert-base-uncased or 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
    
    fine-tunes Bert model using k-old cross validation
    :param sentences: list or array-like. list of texts to classify
    :param labels: list or array-like. list of labels corresponding to sentences
    :param BERT_tokenizer: string, default 'bert-base-uncased'. BERT base model used
    :param n_splits: integer, default None. number of folds to use
    :param random_state: integer, default 42. random seed to initialize folds generation
    :param n_epochs: integer, default 5. number of epochs used to fine-tune Bert
    :param output_dir: string, default None. directory where to save the fine-tuned model
    :param MAX_TKN_LEN: integer, default 511. see https://github.com/huggingface/transformers/issues/2446
    :return:    model: fine-tuned Bert model with highest performance over k folds
                stats: high level performance statistics
                stats_classes: detailed performance statistics
    """
    prep_data = prep_BERT_dataset(sentences=sentences, labels=labels, BERT_tokenizer=BERT_tokenizer,
                                  MAX_TKN_LEN=MAX_TKN_LEN)
    dataset = prep_data['dataset']
    res = run_KFOLD(dataset=dataset, base_model=BERT_tokenizer, model_trainer=run_BERT,
                    base_model_loader=BertForSequenceClassification.from_pretrained,
                    n_splits=n_splits, random_state=random_state, n_epochs=n_epochs, num_labels=prep_data['num_labels'])
    if output_dir is not None:
        try:
            print('saving model in:', output_dir)
            res['model'].save_pretrained(output_dir)
            res['stats_classes'].to_csv(output_dir + '/stats.csv', header=True)
        except:
            print('model not saved, please enter valid path')
    return res

In [ ]:
#Diana's code - can do the same for precision ci and recall ci using precision_score and recall_score (first import them) instead of f1_score

def f1_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    f1 = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        f1[bs] = (f1_score(tp_bootstrap["t"], tp_bootstrap["p"], average='weighted')) #added , average='weighted'
    res = [f1.mean(), np.quantile(f1,0.025), np.quantile(f1,0.975), f1.std()]
    return res

def precision_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    p = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        p[bs] = (precision_score(tp_bootstrap["t"], tp_bootstrap["p"], average='weighted')) #added , average='weighted'
    res = [p.mean(), np.quantile(p,0.025), np.quantile(p,0.975), p.std()]
    return res
  
def recall_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (recall_score(tp_bootstrap["t"], tp_bootstrap["p"], average='weighted')) #added , average='weighted'
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

def class_0_f1_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (f1_score(tp_bootstrap["t"], tp_bootstrap["p"], pos_label=0, average='binary'))
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

def class_1_f1_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (f1_score(tp_bootstrap["t"], tp_bootstrap["p"], pos_label=1, average='binary'))
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

def class_0_precision_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (precision_score(tp_bootstrap["t"], tp_bootstrap["p"], pos_label=0, average='binary'))
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

def class_1_precision_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (precision_score(tp_bootstrap["t"], tp_bootstrap["p"], pos_label=1, average='binary'))
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

def class_0_recall_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (recall_score(tp_bootstrap["t"], tp_bootstrap["p"], pos_label=0, average='binary'))
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

def class_1_recall_conf_int(target, predict, bootstrap_n = 500):
    n = len(target)
    r = np.zeros(bootstrap_n)
    for bs in np.arange(bootstrap_n):
        i = np.random.choice(n, n, replace = True)
        tp = pd.DataFrame({"t": target, "p": predict})
        tp_bootstrap = tp.iloc[i, :]
        r[bs] = (recall_score(tp_bootstrap["t"], tp_bootstrap["p"], pos_label=1, average='binary'))
    res = [r.mean(), np.quantile(r,0.025), np.quantile(r,0.975), r.std()]
    return res

In [ ]:
# fine tune Bert - update output_dir or say None
bert_model = train_BERT(sentences=data_for_classifier['Text_snippet'].to_list(), labels=data_for_classifier['label'].to_list(), test_size=0.2, n_epochs=20, output_dir='./bert_base_v2_Jun26/')
bert_model['stats']

In [ ]:
bert_model

In [ ]:
# run k-fold validation
kf = BERT_KFOLD(sentences=data_for_classifier['Text_snippet'].to_list(), labels=data_for_classifier['label'].to_list(), n_splits=5, BERT_tokenizer=BERT_tokenizer, n_epochs=2,
                random_state=666)
print(kf['stats'], '\n\n', kf['stats_classes'])

In [ ]:
kf['stats_classes'].to_csv('./bert_base_v2_Jun26/kf_stats_classes.csv')

In [ ]:
kf['stats'].to_csv('./bert_base_v2_Jun26/kf_stats.csv')

confidence intervals - this wont work here because the labels and pred variables are from the second bert method below

In [ ]:
model = BertForSequenceClassification.from_pretrained('./bert_base_v2_Jun26')

In [ ]:
testing_data.head()

In [ ]:
#performance on test set


# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(testing_data.shape[0]))

# Create sentence and label lists
sentences = testing_data.Text_snippet.values
labels = testing_data.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 105,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 16  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

# Load the BERT model from a directory path
model_path = "./bert_base_v2_Jun26"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Move model to the desired device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assuming you have your data in some format, load it here
# Example:
# test_data = load_test_data()

# Assuming your data is in a DataFrame format with 'text' and 'label' columns
# You can adjust this based on your actual data format
# texts = test_data['text'].tolist()
# labels = test_data['label'].tolist()
texts = testing_data['Text_snippet'].tolist()
labels = testing_data['label'].tolist()

# Function to generate predictions using the BERT model
def generate_predictions(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the same device as the model
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1)
    return predicted_labels.cpu().numpy()

# Assuming you have loaded your test data and converted it into the required format
# You can then generate predictions using the BERT model
predicted_labels = generate_predictions(texts)

# Assuming you have ground truth labels for your test data
# You can then use the f1_conf_int function to calculate confidence intervals
confidence_intervals = f1_conf_int(labels, predicted_labels)

# Example usage:
print("F1 score:", confidence_intervals[0])
print("95% CI lower bound:", confidence_intervals[1])
print("95% CI upper bound:", confidence_intervals[2])
print("Standard deviation:", confidence_intervals[3])


In [ ]:
# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

In [ ]:
f1_conf_int(labels, predicted_labels)

In [ ]:
f1_conf_int(flat_true_labels, flat_predictions) #gives mean, lower ci, upper ci, std

In [ ]:
f1_score(labels, predicted_labels,average='weighted')

In [ ]:
confidence_intervals = class_0_f1_conf_int(labels, predicted_labels)

print("F1 score:", confidence_intervals[0])
print("95% CI lower bound:", confidence_intervals[1])
print("95% CI upper bound:", confidence_intervals[2])
print("Standard deviation:", confidence_intervals[3])

In [ ]:
confidence_intervals = class_1_f1_conf_int(labels, predicted_labels)

print("F1 score:", confidence_intervals[0])
print("95% CI lower bound:", confidence_intervals[1])
print("95% CI upper bound:", confidence_intervals[2])
print("Standard deviation:", confidence_intervals[3])

In [ ]:
precision_confidence_intervals = precision_conf_int(labels, predicted_labels)

print("Precision:", precision_confidence_intervals[0])
print("95% CI lower bound:", precision_confidence_intervals[1])
print("95% CI upper bound:", precision_confidence_intervals[2])
print("Standard deviation:", precision_confidence_intervals[3])

In [ ]:
precision_confidence_intervals = class_0_precision_conf_int(labels, predicted_labels)

print("Precision:", precision_confidence_intervals[0])
print("95% CI lower bound:", precision_confidence_intervals[1])
print("95% CI upper bound:", precision_confidence_intervals[2])
print("Standard deviation:", precision_confidence_intervals[3])

In [ ]:
precision_confidence_intervals = class_1_precision_conf_int(labels, predicted_labels)

print("Precision:", precision_confidence_intervals[0])
print("95% CI lower bound:", precision_confidence_intervals[1])
print("95% CI upper bound:", precision_confidence_intervals[2])
print("Standard deviation:", precision_confidence_intervals[3])

In [ ]:
recall_confidence_intervals = recall_conf_int(labels, predicted_labels)

print("Recall:", recall_confidence_intervals[0])
print("95% CI lower bound:", recall_confidence_intervals[1])
print("95% CI upper bound:", recall_confidence_intervals[2])
print("Standard deviation:", recall_confidence_intervals[3])

In [ ]:
recall_confidence_intervals = class_0_recall_conf_int(labels, predicted_labels)

print("Recall:", recall_confidence_intervals[0])
print("95% CI lower bound:", recall_confidence_intervals[1])
print("95% CI upper bound:", recall_confidence_intervals[2])
print("Standard deviation:", recall_confidence_intervals[3])

In [ ]:
recall_confidence_intervals = class_1_recall_conf_int(labels, predicted_labels)

print("Recall:", recall_confidence_intervals[0])
print("95% CI lower bound:", recall_confidence_intervals[1])
print("95% CI upper bound:", recall_confidence_intervals[2])
print("Standard deviation:", recall_confidence_intervals[3])

In [ ]:
print(classification_report(labels, predicted_labels))

In [ ]:
bert_report = classification_report(flat_true_labels, flat_predictions, output_dict=True) 
df_bert = pd.DataFrame(bert_report).transpose()
df_bert.to_csv('./bert_base_Jun26/classification_report.csv', index= True)
df_bert.head()

In [ ]:
data_for_classifier.head()

In [ ]:
testing_data.describe()

In [ ]:
testing_data.head()

In [ ]:
test = pd.read_csv('test_data.csv')
test.head(2)

In [ ]:
# RUN PRETRAINED BERT AND ANALYSE ERRORS 
BERT_tokenizer = 'bert-base-uncased' #or can be 'bert-base-uncased'
# model_path = data_path+'bert_models/'+dom+'_biobert'
model_path = './bert_base_v2_Jun26' #update accordingly
#text_col = 'clean_text'
text_col = 'Text_snippet'

#df = pd.read_excel(data_path+'/cognition_adjudicated_clean.xlsx')
df = pd.read_csv('test_data.csv', usecols=['brcid', 'CN_Doc_ID', 'Text_snippet','label' ])
                 #columns=['Brcid', 'CN_Doc_ID','doc_date', 'doc_type', 'kw', 'domain', 'start_kw', 'clean_text'])
# df = df.loc[df.domain == dom].reset_index()
df[text_col] = df[text_col].apply(lambda x: str(x).strip().lower())

res= load_and_run_BERT(sentences=df[text_col]
                       , trained_bert_model=model_path
                       , BERT_tokenizer=BERT_tokenizer)


res.to_csv('./bert_base_v2_Jun26_testdata.csv')

In [ ]:
cpt=0
BERT_tokenizer = 'bert-base-uncased' #'bert-base-uncased'
#BERT_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased') #'bert_base_uncased' #this seems to give a url error? and link that asks to sign in to huggingface
#model_path = data_path+'/bert_models/cognition_all_biobert_rel10'
model_path = './bert_base_v2_Jun26' #'./bert_base_model_nov14_second_script_notoverfit'
#names=['BrcId', 'CN_Doc_ID','doc_date', 'doc_type', 'kw', 'domain', 'start_kw', 'clean_text']#, 'age', 'ethnicitycleaned', 'Gender_ID']
names=['brcid', 'CN_Doc_ID', 'Text_snippet','label']
for chunk in pd.read_csv('./test_data.csv'
                         , chunksize=10000): #, names=names):
    cpt+=1
    print('chunk nb',cpt)
    res= load_and_run_BERT(sentences=chunk['Text_snippet'] #['clean_text']
                   , trained_bert_model=model_path
                   , BERT_tokenizer=BERT_tokenizer)

    result = pd.concat([res.reset_index()[['preds','probs']]
                        , chunk.reset_index()[['brcid', 'CN_Doc_ID', 'Text_snippet', 'label']]] #'BrcId', 'CN_Doc_ID','doc_date', 'doc_type', 'kw', 'domain', 'start_kw', 'clean_text']]]
                       , axis=1)
    print(result.head(1))
    header_bool=1 if cpt==1 else False
    result[['preds','probs']+names].to_csv('./test_data_bert_base_v2_Jun26.csv', mode='a', header=header_bool)

In [ ]:
result.head()

In [ ]:
df.head()

In [ ]:
df.to_csv('test_with_clean_text.csv')